240520 현재 기업 구인 세부사항 크롤링 구문 완성...(오류시 예외문까지) 데이터 프레임 만들고 CSV파일 저장 후 데이터 적재구문 만들기. 이후 검색 조건 변경 계획

## Today's mission

- 잡코리아 데이터 수집
- 잡코리아 셀레니움 웹크롤링   

# Jobkorea Crawling(Selenium) _ Project(No.1)

Today's mission
데이터 수집
셀레니움 웹크롤링

## 셀레니움 : 동적 크롤링

좀 더 복잡한 형태의 데이터를 크롤링하기 위한 동적 크롤링 및 정규 표현식의 사용방법

**참고 사이트**
- https://kihome15.tistory.com/10
- https://coding-shop.tistory.com/290?category=1235575
- https://coding-shop.tistory.com/286?category=1235575


### 동적 크롤링이란?

- 일반적인 크롤링으로는 정적 데이터, 즉 변하지 않는 데이터만을 수집할 수 있다. 
- 한 페이지 안에서 원하는 정보가 모두 드러나는 것을 정적 데이터라 한다. 

- 반면 입력, 클릭, 로그인 등을 통해 데이터가 바뀌는 것을 동적 데이터라 한다. 
- 예를 들어 네이버 지도에서 매장을 검색을 한 후 좌측에서 원하는 선택할 때 마다 이에 해당하는 내용이 뜬다. 

이는 웹페이지에서 사용자가 클릭 등과 같은 조작을 하면 AJAX 호출이 발생하여 그 결과가 페이지의 일부분에만 반영되어 변경되기 때문이다. 즉 매장을 클릭하면 웹브라우저가 연결된 자바스크립트 코드를 실행하여 해당 매장의 상세 정보가 동일한 페이지에 동적으로 표시된다. 정적 페이지와 동적 페이지의 작동 방식의 차이를 나타낸다.

셀레니움을 이용할 경우 정적 페이지와 동적 페이지를 모두 크롤링 할 수 있다는 강력함이 있지만, 상대적으로 속도가 느리다. 따라서 정적 페이지는 기존의 방법을 이용한 크롤링을, 동적 페이지는 셀레니움을 이용한 크롤링을 하는 것이 일반적이다.

- 정적 크롤링과 동적 크롤링 비교
:name: crawl_compare
| 구분 | 정적 크롤링 | 동적 크롤링 |
| --- | --- | --- |
|사용 패키지	|requests |	selenium |
|수집 커버리지 |정적 페이지 | 정적/동적 페이지 |
| 수집 속도 | 빠름 (별도 페이지 조작 필요 X) | 상대적으로 느림 |
| 파싱 패키지 | beautifulsoup | beautifulsoup / selenium |
```

셀레니움이란 다양한 브라우저(인터넷 익스플로러, 크롬, 사파리 오페라 등) 및 플랫폼에서 웹 응용 프로그램을 테스트할 수 있게 해주는 라이브러리다. 즉 웹 자동화 테스트 용도로 개발이 되었기에 실제 브라우저를 사용하며, 페이지가 변화하는 것도 관찰이 가능하기에 동적 크롤링에 사용할 수 있다.

In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
# #### 패키지 먼저 설치 ######
!pip install webdriver_manager

In [3]:
# Chrome 웹 드라이버 다운로드 및 설치 후 실행
# 설치한 selenium에서 webdriver를 import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup

In [4]:
from selenium.common.exceptions import NoSuchElementException

In [5]:
# selenum의 webdriver에 앞서 설치한 chromedirver를 연동한다.
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

`webdriver.Chrome(service=Service(ChromeDriverManager().install()))` 코드를 실행하면 크롬 브라우저의 버전을 탐색한 다음, 버전에 맞는 웹드라이버를 다운로드하여 해당 경로를 셀레니움에 전달해준다. 또한 `selenium_open`와 같이 크롬 창이 열리며, 좌측 상단에 'Chrome이 자동화된 테스트 소프트웨어에 의해 제어되고 있습니다.'라는 문구가 뜬다. 이제 파이썬 코드를 이용해 해당 페이지를 조작할 수 있다. 

In [6]:
# 접속 사이트 주소 인식
url = 'https://www.jobkorea.co.kr/recruit/joblist?menucode=duty'
driver.get(url)
driver.page_source[1:1000]

'html lang="ko"><head>\n    <meta charset="utf-8">\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge; chrome=1">\n    <title>직무별 채용정보 | 잡코리아</title>\n\n\n    \n\n\n\n\n\n<meta name="writer" content="잡코리아">\n<meta name="description" content="직무와 관련된 채용속보, 헤드헌팅 채용공고 등 모든 채용공고를 한눈에 확인하세요!">\n<meta name="keywords" content="경력, 신입, 인턴, 공채, 채용, 구직, 공고, 취업사이트, 취업포털, 채용정보, 대기업, 계열사, 그룹사, 정규직, 1000대기업, 공기업, 신입공채, 잡코리아">\n\n<meta name="verify-v1" content="wfOOCE9Vtx+Z5etOXJnS9LU03yGpBxkK74T/yU63Xqs=">\n\n<meta property="og:site_name" content="잡코리아">\n<meta property="og:title" content="직무별 채용정보 | 잡코리아">\n<meta property="og:description" content="직무와 관련된 채용속보, 헤드헌팅 채용공고 등 모든 채용공고를 한눈에 확인하세요!">\n<meta property="og:image" content="//file1.jobkorea.co.kr/banner/share/share_img_01.png">\n<meta property="og:url" content="https://www.jobkorea.co.kr/recruit/joblist">\n<meta property="og:type" content="website">\n<meta property

`driver.get()` 내에 URL 주소를 입력하면 해당 주소로 이동한다. 또한 `driver.page_source`를 통해 열려있는 창의 HTML 코드를 확인할 수도 있다. 이제 잡코리아 메인에서 채용정보의 '산업별' 버튼을 누르는 동작을 실행해보자. 개발자도구 화면을 통해 확인해보면 '산업별'탭은 아래 HTML에 위치하고 있다.

In [7]:
# 크롤링 할 직무 대단위 선택
driver.find_element(By.XPATH, value = '//*[@id="devSearchForm"]/div[2]/div/div[1]/dl[1]/dd[2]/div[2]/dl[1]/dd/div[1]/ul/li[6]/label/span/span').click()

- 브라우저 상에서 보이는 버튼, 검색창, 사진, 테이블, 동영상 등을 엘레먼트(element, 요소)라고 한다. 
- `find_element()`는 다양한 방법으로 엘레먼트에 접근하게 해주며, 
- `By.*` 를 통해 어떠한 방법으로 엘레먼트에 접근할지 선언한다. 
- LINK_TEXT의 경우 링크가 달려 있는 텍스트로 접근하며, 
- `value = '뉴스'`, 즉 뉴스라는 단어가 있는 엘레먼트로 접근한다. 
- `click()` 함수는 마우스 클릭을 실행하며 결과 적으로 뉴스 탭을 클릭한 후 페이지가 이동
- `find_element()` 내 접근방법 및 셀레니움의 각종 동작 제어 방법에 대해서는 나중에 다시 정리하도록 한다.

이제 해당 탭을 닫아보자.

In [8]:
# 크롤링 할 세부 직무 소단위 다중 선택
for i in range(19):
    if i >= 1 and i <= 3 or i >= 6 and i <= 8 or i == 13:
        # 각 요소에 대한 XPath 문자열 생성
        xpath = f'//*[@id="duty_step2_10031_ly"]/li[{i + 1}]/label/span/span'
        # XPath를 사용하여 해당 요소를 클릭
        driver.find_element(By.XPATH, value=xpath).click()
        print(xpath)
    else:
        pass

//*[@id="duty_step2_10031_ly"]/li[2]/label/span/span
//*[@id="duty_step2_10031_ly"]/li[3]/label/span/span
//*[@id="duty_step2_10031_ly"]/li[4]/label/span/span
//*[@id="duty_step2_10031_ly"]/li[7]/label/span/span
//*[@id="duty_step2_10031_ly"]/li[8]/label/span/span
//*[@id="duty_step2_10031_ly"]/li[9]/label/span/span
//*[@id="duty_step2_10031_ly"]/li[14]/label/span/span


In [9]:
# 검색 버튼 실행
driver.find_element(By.XPATH, value = '//*[@id="dev-btn-search"]/span').click()

# 전체 반복문 기본 구문

title_list=[]      # 제목 리스트
career_list = []   # 경력 리스트
edu_list = []      # 학력 리스트
em_type_list = []  # 고용형태 리스트
pay_list = []      # 급여 리스트
address_list = []  # 근무지 리스트


# 다음페이지 클릭 실행
repeat2=1
while True:
    repeat2 = repeat2 + 1
    time.sleep(1)
    # 페이지 내 회사 선택 for문 실행
    for repeat1 in range(1,40):
        if repeat1 == 2 :
            continue
        else:
            try:
                # 회사 선택
                element = driver.find_element(By.XPATH, value=f'//*[@id="dev-gi-list"]/div/div[5]/table/tbody/tr[{repeat1}]/td[2]/div/strong')
                element.click()
            except NoSuchElementException:
                continue
                time.sleep(3)
            # 새창으로 커서 옮김
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(2)
            # 필요값 크롤링 & 리스트업
            title = driver.find_element(By.XPATH, value = '//*[@id="container"]/section/div[1]/article/div[1]/h3').text.split('\n')[-1]
            time.sleep(0.5)
            career = driver.find_element(By.XPATH, value = '//*[@id="container"]/section/div[1]/article/div[2]/div[1]/dl/dd[1]').text
            time.sleep(0.5)
            edu = driver.find_element(By.XPATH, value = '//*[@id="container"]/section/div[1]/article/div[2]/div[1]/dl/dd[2]').text
            time.sleep(0.5)
            em_type = driver.find_element(By.XPATH, value = '//*[@id="container"]/section/div[1]/article/div[2]/div[2]/dl/dd[1]/ul/li').text
            time.sleep(0.5)
            pay = driver.find_element(By.XPATH, value = '//*[@id="container"]/section/div[1]/article/div[2]/div[2]/dl/dd[2]').text
            time.sleep(0.5)
            address = driver.find_element(By.XPATH, value = '//*[@id="container"]/section/div[1]/article/div[2]/div[2]/dl/dd[3]/a').text
            time.sleep(2)
            title_list.append(title)
            career_list.append(career)
            edu_list.append(edu)
            em_type_list.append(em_type)
            pay_list.append(pay)
            address_list.append(address)
            time.sleep(3)
            # 창 닫기
            driver.close()
            time.sleep(2)
            # 원래 창으로 커서 옮김
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(2)
    # 페이지 내 검색 끝날 경우
    if repeat2 > 1 and repeat2 < 11:
        # 다음 페이지 클릭
        driver.find_element(By.XPATH, value = f'//*[@id="dvGIPaging"]/div/ul/li[{repeat2}]/a').click()
        time.sleep(3)
    # 10개 페이지 끝날 경우
    elif repeat2 == 11:
        # 다음 10개 페이지 중 첫번째 페이지로 이동
        driver.find_element(By.XPATH, value = '//*[@id="dvGIPaging"]/div/p/a').click()
        repeat2=0
        time.sleep(5)

In [10]:
# 전체 반복문 오류 예외 뜨는 경우 종료후 재실행

title_list=[]      # 제목 리스트
career_list = []   # 경력 리스트
edu_list = []      # 학력 리스트
em_type_list = []  # 고용형태 리스트
pay_list = []      # 급여 리스트
address_list = []  # 근무지 리스트
repeat2=1          # 10개 페이지 내 쪽수 순서
rotate=0           # 10개 페이지 경과 횟수
order=0            # 1개 페이지 내 검색 순서
done=False         # 오류 발생시 기존 구동 연결 변수
page_num = 1       # 오류 발생시 기존 구동 연결 변수 2


while True:
    try:
        if done == False:
            repeat2 = repeat2 + 1
            time.sleep(1)
            page_num=1
            # 페이지 내 회사 선택 for문 실행(repeat1 : 1 페이지 내 검색 순서)
            for repeat1 in range(page_num,40):
                if repeat1 == 2 :
                    continue
                else:
                    try:
                        # 회사 선택
                        element = driver.find_element(By.XPATH, value=f'//*[@id="dev-gi-list"]/div/div[5]/table/tbody/tr[{repeat1}]/td[2]/div/strong')
                        element.click()
                    except NoSuchElementException:
                        continue
                        time.sleep(3)
                    # 새창으로 커서 옮김
                    driver.switch_to.window(driver.window_handles[1])
                    time.sleep(2)
                    # 필요값 크롤링 & 리스트업
                    title = driver.find_element(By.XPATH, value = '//*[@id="container"]/section/div[1]/article/div[1]/h3').text.split('\n')[-1]
                    time.sleep(0.5)
                    career = driver.find_element(By.XPATH, value = '//*[@id="container"]/section/div[1]/article/div[2]/div[1]/dl/dd[1]').text
                    time.sleep(0.5)
                    edu = driver.find_element(By.XPATH, value = '//*[@id="container"]/section/div[1]/article/div[2]/div[1]/dl/dd[2]').text
                    time.sleep(0.5)
                    em_type = driver.find_element(By.XPATH, value = '//*[@id="container"]/section/div[1]/article/div[2]/div[2]/dl/dd[1]/ul/li').text
                    time.sleep(0.5)
                    pay = driver.find_element(By.XPATH, value = '//*[@id="container"]/section/div[1]/article/div[2]/div[2]/dl/dd[2]').text
                    time.sleep(0.5)
                    address = driver.find_element(By.XPATH, value = '//*[@id="container"]/section/div[1]/article/div[2]/div[2]/dl/dd[3]/a').text
                    time.sleep(2)
                    title_list.append(title)
                    career_list.append(career)
                    edu_list.append(edu)
                    em_type_list.append(em_type)
                    pay_list.append(pay)
                    address_list.append(address)
                    time.sleep(3)
                    # 창 닫기
                    driver.close()
                    time.sleep(2)
                    # 원래 창으로 커서 옮김
                    driver.switch_to.window(driver.window_handles[0])
                    time.sleep(2)
                    order = order + 1
                    if page_num == 40:
                        done=False
            # 페이지 내 검색 끝날 경우
            if repeat2 > 1 and repeat2 < 11:
                # 다음 페이지 클릭
                driver.find_element(By.XPATH, value = f'//*[@id="dvGIPaging"]/div/ul/li[{repeat2}]/a').click()
                order = 0
                time.sleep(3)
            # 10개 페이지 끝날 경우
            elif repeat2 == 11:
                # 다음 10개 페이지 중 첫번째 페이지로 이동
                driver.find_element(By.XPATH, value = '//*[@id="dvGIPaging"]/div/p/a').click()
                rotate = totate + 1
                order = 0
                repeat2=0
                time.sleep(5)
    except:
        driver.quit()
        time.sleep(2)
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        time.sleep(2)
        url = 'https://www.jobkorea.co.kr/recruit/joblist?menucode=duty'
        driver.get(url)
        time.sleep(5)
        driver.find_element(By.XPATH, value = '//*[@id="devSearchForm"]/div[2]/div/div[1]/dl[1]/dd[2]/div[2]/dl[1]/dd/div[1]/ul/li[6]/label/span/span').click()
        for i in range(19):
            if i >= 1 and i <= 3 or i >= 6 and i <= 8 or i == 13:
                # 각 요소에 대한 XPath 문자열 생성
                xpath = f'//*[@id="duty_step2_10031_ly"]/li[{i + 1}]/label/span/span'
                # XPath를 사용하여 해당 요소를 클릭
                driver.find_element(By.XPATH, value=xpath).click()
                print(xpath)
            else:
                pass
        # 검색 버튼 실행
        driver.find_element(By.XPATH, value = '//*[@id="dev-btn-search"]/span').click()
        done =True
        page_num = repeat1
        # 진행중인 페이지 이동
#         for a in len(range(rotate)):
#             driver.find_element(By.XPATH, value = '//*[@id="dvGIPaging"]/div/p/a').click()
#         driver.find_element(By.XPATH, value = f'//*[@id="dvGIPaging"]/div/ul/li[{repeat2}]/a').click()

KeyboardInterrupt: 

In [11]:
title_list

['[병역특례-보충역] 연구개발 직군 (메타버스)',
 '[병역특례-보충역]JAVA 개발자 채용',
 '써모피셔 사이언티픽 코리아 5월 4주차 신입/경력 채용',
 '부문별 경력/신입사원 모집',
 '㈜이엠넷 2024년 상반기 기획자/카피라이터/디자이너/총무/회계 공채 [서울본사]',
 '2024년 경기청년 일자리 매치업 취업지원 사업',
 '2024년도 YC-EXICON-SEMCNS GROUP 상반기 신입사원 공개채용',
 '아이텍솔루션 교육컨텐츠 AI 앱 개발자 채용',
 '2024 상반기 공개채용(마케팅/영업/CS/개발/사업기획/영어강사)',
 '각 부문 채용 [기획/디자인/개발/퍼블리셔]',
 'VA 스타일 프론트앤드 개발자',
 '[토스] QA Specialist',
 'AI 혁신 우수인재 채용 [CLOUD분야]',
 'AI 혁신 우수인재 채용 [DATA분야]',
 'AI 혁신 우수인재 채용 [AI분야]',
 'AI 혁신 우수인재 채용 [IT분야]',
 '이지샵 주식회사 2024년 상반기 채용',
 '[전액국비] 2024년 취업유망직종 교육생 모집(물류/무역/IT/자바/웹개발)',
 '제조공장 설비 시뮬레이션 및 데이터 해석 연구직 신입사원 모집',
 '원스토어 Android SDK 개발자',
 'iOS SDK',
 '2024 기아 5월 경력 채용 모빌리티 부문',
 '2024 기아 5월 경력 채용 생산/품질 부문',
 '2024 기아 5월 경력 채용 IT 부문',
 '[건강기능식품전문] 각 부문 경력 채용',
 '각 분야별 신입경력 모집 (영업부/기술지원부/연구개발부)',
 '반도체자동화설비,장비(Maintenance ) 유지보수 및 전산모니터링 직원 모집',
 '씨알케이㈜ 각부분별 신입/경력직 채용',
 '유능한 웹&앱 디자이너 / 퍼블리셔 / 개발자 경력직(팀장급) 모집요강',
 '탄소회계 개발 Head Engineer',
 '머신러닝 개발자 (DL/ML 기반 데이터분석)',
 '플랫폼 개발 프론트앤드 개발자',
 '디아이 ERP 개발 경력 사원

In [12]:
career_list

['신입',
 '신입',
 '신입·경력',
 '경력무관',
 '신입·경력 직무에 따라 상이',
 '신입·경력 모집분야별 상이',
 '신입',
 '경력 (5년이상)',
 '신입·경력',
 '경력 (3년이상) 상세요강 참조 (부서마다 상이)',
 '경력 (1년이상)',
 '경력무관',
 '경력 (3년이상) 직무별상이',
 '신입·경력 (3년이상) 직무별상이',
 '경력 (3년이상) 직무별상이',
 '신입·경력 (3년이상) 직무별상이',
 '신입·경력',
 '신입',
 '경력무관',
 '경력 (3년이상)',
 '경력 (5년이상)',
 '경력',
 '경력',
 '경력',
 '경력 (1년이상)',
 '신입·경력',
 '신입·경력',
 '신입·경력 (3년이상)',
 '경력 (5년이상) 부서별 상이',
 '경력 (5년이상)',
 '경력 (3년이상)',
 '경력 (3년이상)',
 '경력 (4년이상)',
 '신입',
 '신입·경력 (4년이상)',
 '경력 (5년이상) UX리서치 최소 3년 이상',
 '신입·경력',
 '경력 (3년이상) 직무에 따라 상이',
 '경력 (7년이상)',
 '경력 (3년이상)',
 '신입·경력',
 '신입·경력',
 '신입·경력',
 '경력 (3년이상)',
 '신입·경력',
 '신입·경력 (3년이상) 직무에 따라 상이',
 '경력 (5년이상)',
 '경력',
 '신입·경력 (1년이상) 직무에 따라 상이',
 '신입·경력',
 '경력',
 '신입·경력 (3년이상) 직무별상이',
 '경력무관',
 '신입·경력 (1년이상) 부서별 경력년수 : 6개월~1년이상',
 '신입·경력 (2년이상) 직무별 상이',
 '신입·경력 (5년이상) PC기반 개발/운영(신입/경력5년이내) /개발직무(5년~10년미만)',
 '경력 (1년이상) 직무별상이',
 '신입·경력',
 '경력 (3년이상) 직무별상이',
 '경력 (2년이상)',
 '경력 (2년이상)',
 '경력 (2년이상)',
 '신입·경력',
 '경력 (3년이상)',
 '신입·경력 3년미만',
 '신입·경력',
 '신

In [13]:
edu_list

['대졸이상',
 '대졸이상',
 '학력무관',
 '초대졸이상',
 '대졸이상 (졸업예정자 가능)',
 '학력무관 모집분야별 상이',
 '대졸이상 (졸업예정자 가능)',
 '학력무관',
 '학력무관',
 '학력무관',
 '대졸이상',
 '학력무관',
 '대졸이상',
 '대졸이상',
 '대졸이상',
 '대졸이상',
 '대졸이상',
 '학력무관',
 '대졸이상 (졸업예정자 가능)',
 '대졸이상',
 '대졸이상',
 '대졸이상',
 '대졸이상',
 '고졸이상',
 '고졸이상',
 '학력무관 분야별 상이',
 '고졸이상',
 '초대졸이상',
 '초대졸이상',
 '대졸이상',
 '대졸이상',
 '대졸이상',
 '학력무관',
 '학력무관',
 '학력무관 부문별 상이',
 '대졸이상',
 '학력무관',
 '대졸이상 직무에 따라 상이',
 '학력무관',
 '대졸이상',
 '초대졸이상',
 '고졸이상 직무별 상이',
 '초대졸이상 (졸업예정자 가능)',
 '초대졸이상',
 '고졸이상',
 '학력무관',
 '학력무관',
 '학력무관',
 '대졸이상',
 '초대졸이상',
 '학력무관',
 '초대졸이상',
 '학력무관',
 '학력무관',
 '대졸이상 (졸업예정자 가능)',
 '대졸이상',
 '대졸이상',
 '학력무관',
 '학력무관',
 '학력무관',
 '학력무관',
 '학력무관',
 '학력무관',
 '대졸이상',
 '초대졸이상',
 '대졸이상 (졸업예정자 가능)',
 '학력무관 모집분야별 상이',
 '학력무관 분야별 상이',
 '고졸이상 직무별 상이',
 '초대졸이상',
 '초대졸이상 직무별상이',
 '초대졸이상',
 '학력무관',
 '대졸이상',
 '학력무관',
 '대졸이상 (졸업예정자 가능)',
 '학력무관',
 '학력무관',
 '대졸이상',
 '초대졸이상',
 '학력무관',
 '학력무관',
 '대졸이상',
 '학력무관',
 '학력무관',
 '학력무관']

In [14]:
em_type_list

['정규직 수습 3개월',
 '정규직 수습 3개월',
 '정규직',
 '정규직 수습 3개월',
 '정규직',
 '정규직 수습 2개월',
 '정규직',
 '정규직',
 '정규직 수습 3개월',
 '정규직',
 '정규직',
 '계약직 근무기간 12개월',
 '정규직',
 '정규직',
 '정규직',
 '정규직',
 '정규직 수습 3개월',
 '정규직',
 '정규직 수습 3개월',
 '정규직 수습 3개월',
 '정규직 수습 3개월',
 '정규직',
 '정규직',
 '정규직',
 '정규직',
 '정규직 수습기간협의',
 '정규직',
 '정규직',
 '정규직 수습 3개월',
 '정규직',
 '정규직',
 '정규직',
 '정규직',
 '인턴 정규직 전환가능',
 '정규직',
 '정규직',
 '정규직',
 '정규직',
 '정규직',
 '정규직',
 '정규직 수습 3개월',
 '정규직',
 '정규직 수습 3개월',
 '정규직',
 '정규직 수습 3개월',
 '정규직',
 '정규직',
 '정규직 수습 6개월',
 '정규직',
 '정규직',
 '계약직 근무기간 1년 , 직무별 상이 /근무평가 후 갱신가능',
 '정규직',
 '정규직 수습 3개월',
 '정규직 수습 3개월',
 '정규직 수습기간협의 , 자세한 사항은 면접시 설명',
 '정규직',
 '정규직 수습 3개월',
 '정규직 수습 3개월',
 '정규직',
 '정규직',
 '정규직',
 '정규직',
 '정규직',
 '정규직',
 '정규직',
 '정규직',
 '정규직',
 '정규직',
 '인턴 근무기간 4개월',
 '정규직 수습 3개월',
 '정규직 수습 6개월',
 '정규직',
 '정규직',
 '정규직',
 '정규직',
 '인턴',
 '정규직 수습 3개월',
 '정규직 수습 3개월',
 '정규직',
 '정규직',
 '정규직 수습 3개월',
 '정규직',
 '정규직 수습 3개월',
 '정규직 수습 3개월',
 '정규직 수습 3개월',
 '정규직']

In [15]:
pay_list

['회사내규에 따름',
 '회사내규에 따름',
 '회사내규에 따름',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름',
 '연봉 5,000만원 이상 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름',
 '회사내규에 따름',
 '회사내규에 따름',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '연봉 3,600~3,600만원',
 '회사내규에 따름',
 '회사내규에 따름',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름',
 '회사내규에 따름 - 면접 후 결정',
 '연봉 3,500~4,000만원 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정 ★ 희망 연봉 기재 필수',
 '회사내규에 따름',
 '회사내규에 따름',
 '회사내규에 따름',
 '회사내규에 따름',
 '회사내규에 따름',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '회사내규에 따름 - 면접 후 결정',
 '회사내

In [16]:
address_list

['서울시 강서구',
 '서울시 강서구',
 '경기도 용인시 기흥구',
 '서울시 서초구',
 '서울시 구로구',
 '경기도 부천시',
 '경기도 성남시 분당구',
 '서울시 서초구',
 '서울시 금천구',
 '서울시 강남구',
 '서울시 강남구',
 '서울시 강남구',
 '서울시 종로구',
 '서울시 종로구',
 '서울시 종로구',
 '서울시 종로구',
 '서울시 중구',
 '부산 부산진구',
 '서울시 금천구',
 '경기도 성남시 분당구',
 '경기도 성남시 분당구',
 '서울시 서초구',
 '서울시 서초구',
 '서울시 서초구',
 '경기도 의왕시',
 '서울시 금천구',
 '경기도 수원시 영통구',
 '서울시 영등포구',
 '경기도 시흥시',
 '서울시 용산구',
 '서울시 용산구',
 '서울시 영등포구',
 '서울시 강남구',
 '서울시 금천구',
 '서울시 중구',
 '서울시 강서구',
 '제주도 제주시',
 '서울시 종로구',
 '경기도 화성시',
 '서울시 종로구',
 '서울시 금천구',
 '경기도 화성시',
 '충남 아산시',
 '서울시 서초구',
 '서울시 중구',
 '경기도 성남시 수정구',
 '경기도 의왕시',
 '경기도 성남시 분당구',
 '서울시 마포구',
 '서울시 서초구',
 '경기도 수원시 팔달구',
 '경기도 성남시 중원구',
 '경기도 안양시 동안구',
 '서울시 금천구',
 '서울시 영등포구',
 '서울시 용산구',
 '서울시 강서구',
 '서울시 강남구',
 '서울시 강남구',
 '서울시 송파구',
 '서울시 송파구',
 '서울시 송파구',
 '서울시 마포구',
 '서울시 서초구',
 '인천시 연수구',
 '서울시 강남구',
 '충남 천안시 동남구',
 '경기도 과천시',
 '서울시 성동구',
 '서울시 금천구',
 '서울시 강남구',
 '충북 청주시 흥덕구',
 '경기도 성남시 분당구',
 '서울시 서초구',
 '인천시 남동구',
 '서울시 용산구',
 '서울시 서초구',
 '서울시 서초구',
 '서울시 중구',
 '서울

In [19]:
#!pip install pymysql
import pandas as pd
import pymysql
import numpy as np

In [20]:
data = {'Title':title_list, 'Career':career_list, 'Edu':edu_list, 'Em_type':em_type_list, 'Pay':pay_list, 'Address':address_list}
df = pd.DataFrame(data)
df.head()

,Title,Career,Edu,Em_type,Pay,Address
0,[병역특례-보충역] 연구개발 직군 (메타버스),신입,대졸이상,정규직 수습 3개월,회사내규에 따름,서울시 강서구
1,[병역특례-보충역]JAVA 개발자 채용,신입,대졸이상,정규직 수습 3개월,회사내규에 따름,서울시 강서구
2,써모피셔 사이언티픽 코리아 5월 4주차 신입/경력 채용,신입·경력,학력무관,정규직,회사내규에 따름,경기도 용인시 기흥구
3,부문별 경력/신입사원 모집,경력무관,초대졸이상,정규직 수습 3개월,회사내규에 따름 - 면접 후 결정,서울시 서초구
4,㈜이엠넷 2024년 상반기 기획자/카피라이터/디자이너/총무/회계 공채 [서울본사],신입·경력 직무에 따라 상이,대졸이상 (졸업예정자 가능),정규직,회사내규에 따름 - 면접 후 결정,서울시 구로구


In [ ]:
df.to_csv('data/Jobkorea_crawling_list_LSH.csv', sep=",", encoding='UTF-8')

In [ ]:
#### 아래 실행 전 테이블 삭제
def db_delete():
    import pymysql
    # MariaDB 연결
    conn = pymysql.connect(host='127.0.0.1', user='user', password='user1234', db='dbmaria', charset='utf8')
    # 커서 생성
    cur = conn.cursor()
    # User 테이블 생성 (nameUser에 UNIQUE 제약 추가)
    cur.execute('''DROP TABLE user''')
    # 변경사항 커밋
    conn.commit()
    # 연결 종료
    conn.close()

In [ ]:
##############################
# ChatGPT 활용 및 테이블 생성
##############################
# 파이썬 데이터프레임을 maria db에 적재하기 위한 테이블 생성문을 작성해줘 ... 
# 다음 데이터프레임 정보를 참고해서...
# CREATE TABLE idv_stocks(
# 	user_id VARCHAR(50),
# 	stocks_id VARCHAR(10),
# 	stocks_name VARCHAR(30),
# 	ins_date DATETIME,
# 	PRIMARY KEY (user_id, stocks_id)
# );

# CREATE TABLE IF NOT EXISTS gu_real_estate3 (
#     transaction_id INT AUTO_INCREMENT,
#     transaction_amount VARCHAR(255),
#     transaction_type VARCHAR(255),
#     construction_year VARCHAR(255),
#     year INT,
#     dong VARCHAR(255),
#     registration_date VARCHAR(255),
#     seller VARCHAR(255),
#     buyer VARCHAR(255),
#     legal_dong VARCHAR(255),
#     apartment_name VARCHAR(255),
#     month INT,
#     day INT,
#     exclusive_area FLOAT,
#     broker_location VARCHAR(255),
#     jibun VARCHAR(255),
#     region_code INT,
#     floor INT,
#     cancellation_reason_date VARCHAR(255),
#     cancellation_yn VARCHAR(255),
#     GU_id VARCHAR(255),
#     YM_id VARCHAR(255),
#     PRIMARY KEY (transaction_id, GU_id, YM_id)
# );

In [ ]:
import pymysql
# MariaDB 연결
conn = pymysql.connect(host='127.0.0.1', port=3306, user='user', password='user1234', db='dbmaria', charset='utf8')
# 커서 생성
cur = conn.cursor()
# User 테이블 생성 (nameUser에 UNIQUE 제약 추가)
cur.execute('''CREATE TABLE IF NOT EXISTS user (
                nameUser VARCHAR(255) PRIMARY KEY,
                ageUser INTEGER,
                phoneUser VARCHAR(20)
            )''')
# 삽입할 데이터
data_list = [
    ('user1', 25, '010-0123-4567'),
    ('user2', 30, '010-1234-5678'),
    ('user3', 35, '010-2345-6789'),
    ('user4', 40, '010-3456-7890'),
    ('user5', 66, '010-3676-7230')
]
# 데이터 삽입
# # 데이터 프레임의 데이터를 데이터베이스에 삽입
for index, row in df.iterrows():
    cur.execute("""
        INSERT INTO gu_real_estate3 (
            transaction_amount,
            transaction_type,
            construction_year,
            year,
            dong,
            registration_date,
            seller,
            buyer,
            legal_dong,
            apartment_name,
            month,
            day,
            exclusive_area,
            broker_location,
            jibun,
            region_code,
            floor,
            cancellation_reason_date,
            cancellation_yn,
            GU_id,
            YM_id
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['거래금액'],
        row['거래유형'],
        row['건축년도'],
        row['년'],
        row['동'],
        row['등기일자'],
        row['매도자'],
        row['매수자'],
        row['법정동'],
        row['아파트'],
        row['월'],
        row['일'],
        row['전용면적'],
        row['중개사소재지'],
        row['지번'],
        row['지역코드'],
        row['층'],
        row['해제사유발생일'],
        row['해제여부'],
        row['GU'],
        row['YM']
    ))
cur.executemany(sql, data_list)
# 변경사항 커밋
conn.commit()
# 연결 종료
conn.close()

In [ ]:
import pymysql
conn = pymysql.connect(host='127.0.0.1', user='user', password='user1234', db='dbmaria', charset='utf8')
cur = conn.cursor()
sql = "SELECT * FROM user"
cur.execute(sql)
results = cur.fetchall()
for i in results:
    print(i[0] + " / " + str(i[1]) + " / " + i[2])
conn.close()